# EPAM

In [1]:
import re
import warnings
import pandas as pd

from bs4 import BeautifulSoup

# Switch warning off for `bs4`.
warnings.filterwarnings("ignore", category=UserWarning, module='bs4')

In [2]:
posts_file = 'NLP_dataset_final/Posts.csv'

In [3]:
%time posts_df = pd.read_csv(posts_file, encoding='ISO-8859-1')

CPU times: user 12.5 s, sys: 998 ms, total: 13.5 s
Wall time: 14 s


<string>:2: DtypeWarning: Columns (3,18,19) have mixed types. Specify dtype option on import or set low_memory=False.


In [4]:
def replace_unescape(text, unescape_dict={'&lt;': '<', '&gt;': '>', '&amp;': '&', '\r': ''}):
    for k, v in unescape_dict.items():
        text = text.replace(k, v)
    for k, v in unescape_dict.items():
        text = text.replace(k, v)
    return text


def delete_tag(text, tags={'p'}):
    for tag in tags:
        text = text.replace(f'<{tag}>', '')
        text = text.replace(f'</{tag}>', '')
    return text


def get_text(text, sep='\n', delete_code=True, code_replacer='CODE'):
    # Code.
    soup = BeautifulSoup(text, 'lxml')
    if delete_code:
        for i, tag in enumerate(soup.find_all('code')):
            tag.replaceWith(code_replacer.format(i))
    
    # Main part.
    return sep.join(delete_tag(replace_unescape(p_tag.text)) for p_tag in soup.find_all('p'))

In [5]:
sample = posts_df.Body[0]
sample

'<p>I have data  to export in an xml file.For this reason, I use a <code>DOMDocument</code> object with <code>utf-8</code> encoding.\r\r\r\r\nAll special characters are encoded except apostrophe and double quotes.</p>\r\r\r\r\n\r\r\r\r\n<p><strong>Before export</strong> : <code>&lt;p&gt;"don\'t like"&lt;/p&gt;</code></p>\r\r\r\r\n\r\r\r\r\n<p><strong>After export</strong> : <code>&amp;lt;p&amp;gt;don\'t like &amp;lt;/p&amp;gt;</code> </p>\r\r\r\r\n\r\r\r\r\n<p><strong>My snippet</strong> :</p>\r\r\r\r\n\r\r\r\r\n<pre><code>$xml_file = new \\DOMDocument(\'1.0\', \'utf-8\');\r\r\r\r\n$xml_file-&gt;formatOutput = true;\r\r\r\r\n ....\r\r\r\r\n\r\r\r\r\n$xml_file-&gt;save($file_path);\r\r\r\r\n</code></pre>\r\r\r\r\n\r\r\r\r\n<p>Any suggestions?</p>\r\r\r\r\n'

In [6]:
get_text(sample)

'I have data  to export in an xml file.For this reason, I use a CODE object with CODE encoding.\nAll special characters are encoded except apostrophe and double quotes.\nBefore export : CODE\nAfter export : CODE \nMy snippet :\nAny suggestions?'